<a href="https://colab.research.google.com/github/minghaozou/pytorch-tutorial-with-solutions/blob/main/solutions/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [14]:
import os
os.makedirs("going_modular", exist_ok=True)

In [15]:
# YOUR CODE HERE
%%writefile going_modular/get_data.py

"""
Check if the data already exists and skip downloading if it does
"""
import os
import zipfile

from pathlib import Path
import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
  print(f"{image_path} directory already exists")
else:
  print(f"Did not find {image_path} directory, creating one...")
  image_path.mkdir(parents=True, exist_ok=True)

# Download Pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading pizza, steak, sushi data...")
  f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  print("Unzipping pizza, steak and sushi data...")
  zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

Overwriting going_modular/get_data.py


In [16]:
# Example running of get_data.py
!python going_modular/get_data.py

data/pizza_steak_sushi directory already exists
Unzipping pizza, steak and sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [17]:
# Clone the Python Scripts from Github
!git clone https://github.com/minghaozou/pytorch-tutorial-with-solutions.git

fatal: destination path 'pytorch-tutorial-with-solutions' already exists and is not an empty directory.


In [18]:
# move all python scripts from the cloned folder into the existing going_modular folder
!mv /content/pytorch-tutorial-with-solutions/going_modular/*.py /content/going_modular/

mv: cannot stat '/content/pytorch-tutorial-with-solutions/going_modular/*.py': No such file or directory


In [19]:
# Set the path so Python can import it
import sys
sys.path.append('/content/going_modular')

In [20]:
# YOUR CODE HERE
%%writefile going_modular/train.py
import argparse, os, torch

from torchvision import transforms

import data_setup, engine, model_builder, utils

def get_args():
  parser = argparse.ArgumentParser(description="Train a neural network with custom hyperparameters") # Create a parser object

  parser.add_argument("--num_epochs", type = int, default = 5, help = "Number of epochs to train for")
  parser.add_argument("--batch_size", type = int, default = 32, help = "Batch size for DataLoader")
  parser.add_argument("--hidden_units", type = int, default = 64, help = "Number of hidden units in the model")
  parser.add_argument("--learning_rate", type = float, default=0.001, help = "Learning rate for optimizer")

  args = parser.parse_args()
  return args

if __name__ == "__main__":
  args = get_args()
  print(f"Training configuration")
  print(f"\nEpochs: {args.num_epochs}")
  print(f"\nBatch size: {args.batch_size}")
  print(f"\nHidden units: {args.hidden_units}")
  print(f"\nLearning rate: {args.learning_rate}")

  # Setup directories
  train_dir = "data/pizza_steak_sushi/train"
  test_dir = "data/pizza_steak_sushi/test"

  # Setup target device
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # Create transforms
  data_transform = transforms.Compose([
      transforms.Resize((64, 64)),
      transforms.ToTensor()
  ])

  # Create DataLoaders with help from data_setup.py
  train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
      train_dir = train_dir,
      test_dir = test_dir,
      transform = data_transform,
      batch_size = args.batch_size
  )

  #Create model with help from model_builder.py
  model = model_builder.TinyVGG(
      input_shape = 3,
      hidden_units = args.hidden_units,
      output_shape = len(class_names)
  ).to(device)

  # Set loss and optimizer
  loss_fn = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = args.learning_rate)

  # Start training with help from engine.py
  engine.train(
      model = model,
      train_dataloader = train_dataloader,
      test_dataloader = test_dataloader,
      optimizer = optimizer,
      loss_fn = loss_fn,
      epochs = args.num_epochs,
      device = device
  )

  # Save the model with help from utils.py
  utils.save_model(
      model = model,
      target_dir = "models",
      model_name = "05_pytorch_going_modular_exercise_tinyvgg_model.pth"
  )

Overwriting going_modular/train.py


In [21]:
# Example running of train.py
!python going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

Training configuration

Epochs: 5

Batch size: 128

Hidden units: 128

Learning rate: 0.0003
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1032 | train_acc: 0.3524 | test_loss: 1.0931 | test_acc: 0.3333
 20% 1/5 [00:01<00:05,  1.35s/it]Epoch: 2 | train_loss: 1.0866 | train_acc: 0.3613 | test_loss: 1.0703 | test_acc: 0.4933
 40% 2/5 [00:02<00:03,  1.17s/it]Epoch: 3 | train_loss: 1.0708 | train_acc: 0.4485 | test_loss: 1.0472 | test_acc: 0.4800
 60% 3/5 [00:03<00:02,  1.11s/it]Epoch: 4 | train_loss: 1.0290 | train_acc: 0.5454 | test_loss: 1.0396 | test_acc: 0.4000
 80% 4/5 [00:04<00:01,  1.08s/it]Epoch: 5 | train_loss: 0.9796 | train_acc: 0.5312 | test_loss: 0.9986 | test_acc: 0.4667
100% 5/5 [00:05<00:00,  1.10s/it]
[INFO] Saving model to: models/05_pytorch_going_modular_exercise_tinyvgg_model.pth


In general, **the order of command-line arguments in `argparse` does *not* matter**, as long as each argument flag (like `--num_epochs`) is followed by its corresponding value.

---

### ✅ Correct — any order works

All of the following are **equivalent**:

```bash
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003
```

```bash
!python train.py --batch_size 128 --learning_rate 0.0003 --num_epochs 5 --hidden_units 128
```

```bash
!python train.py --learning_rate 0.0003 --hidden_units 128 --num_epochs 5 --batch_size 128
```

`argparse` internally matches flags (`--num_epochs`, `--batch_size`, etc.) to their names, **not** their positions.

---

### ⚠️ The only exceptions

1. **Positional arguments** (no `--`) *do* depend on order.
   For example:

   ```python
   parser.add_argument("train_dir")
   parser.add_argument("test_dir")
   ```

   Then:

   ```bash
   python train.py data/train data/test
   ```

   Here `data/train` → `train_dir`, and `data/test` → `test_dir` — order matters.


The `help` and `description` arguments in **`argparse`** exist purely to make your command-line interface self-documenting and user-friendly. They don’t affect the actual logic of your program — instead, they control what gets printed when someone runs your script with the `-h` or `--help` flag.

Let’s go over this step-by-step 👇

---

### 🧩 1. `description` — overview of your script

You pass `description` when you *create* the parser:

```python
parser = argparse.ArgumentParser(
    description="Train a neural network with custom hyperparameters."
)
```

This text appears at the top of the automatically generated help message and explains what your script does.

---

### ⚙️ 2. `help` — explanation for each argument

You pass `help` inside `add_argument` for each flag:

```python
parser.add_argument("--num_epochs", type=int, default=5,
                    help="Number of epochs to train for")
parser.add_argument("--batch_size", type=int, default=32,
                    help="Number of samples per batch")
```

Each `help` string describes what that particular flag does.

---

### 💡 3. When and how to call it

When a user runs your script with:

```bash
python train.py --help
```

or

```bash
python train.py -h
```

`argparse` automatically prints a neatly formatted usage guide that combines your `description` and all your `help` messages.

---

### 🖥️ Example Output

If your `train.py` contains:

```python
import argparse

parser = argparse.ArgumentParser(description="Train a model on image data.")
parser.add_argument("--num_epochs", type=int, default=5,
                    help="Number of epochs to train for")
parser.add_argument("--batch_size", type=int, default=32,
                    help="Batch size for DataLoader")
parser.add_argument("--learning_rate", type=float, default=0.001,
                    help="Learning rate for the optimizer")

args = parser.parse_args()
```

Then running:

```bash
python train.py --help
```

Will show something like:

```
usage: train.py [-h] [--num_epochs NUM_EPOCHS] [--batch_size BATCH_SIZE]
                [--learning_rate LEARNING_RATE]

Train a model on image data.

options:
  -h, --help            show this help message and exit
  --num_epochs NUM_EPOCHS
                        Number of epochs to train for
  --batch_size BATCH_SIZE
                        Batch size for DataLoader
  --learning_rate LEARNING_RATE
                        Learning rate for the optimizer
```

---


Great — this is one of the most important (and slightly confusing) lines in Python scripts:

```python
if __name__ == "__main__":
```

Let’s break it down clearly 👇

---

### 🧠 1. What it means conceptually

Every Python file has a special built-in variable called `__name__`.

* When you **run a file directly** (e.g., `python train.py`),
  → Python sets `__name__` to `"__main__"`.
* When you **import that file as a module** (e.g., `import train` inside another script),
  → Python sets `__name__` to the module’s name (`"train"` in this example).

So this condition:

```python
if __name__ == "__main__":
```

means

> “Only run the following code if this file is executed directly, not when it’s imported.”

---

### ⚙️ 2. Why it’s useful

It prevents code from running automatically when your file is imported elsewhere.
For example:

```python
# train.py
def train_model():
    print("Training started!")

if __name__ == "__main__":
    print("Running train.py directly")
    train_model()
```

* If you run this:

  ```bash
  python train.py
  ```

  Output:

  ```
  Running train.py directly
  Training started!
  ```

* But if you import it from another script:

  ```python
  import train
  ```

  Output:
  *(nothing prints)* — because the block under `if __name__ == "__main__":` is **skipped**.

---


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [32]:
# YOUR CODE HERE
%%writefile going_modular/predict.py
from torchvision import transforms
from pathlib import Path

import argparse

import torch, torchvision

import model_builder

def get_args():
  parser = argparse.ArgumentParser(description="Predict on a target image given a file path with a saved model")

  parser.add_argument("--image", help = "Path to input image")

  args = parser.parse_args()
  return args

if __name__ == "__main__":
  args = get_args()
  image_path = Path(args.image)
  print(f"Image path:")
  print(f"  Image path: {image_path}")

  # Setup device
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # Create a new instance of TinyVGG
  loaded_model_0 = model_builder.TinyVGG(
      input_shape = 3,
      hidden_units = 128,
      output_shape = 3
  )

  loaded_model_0.load_state_dict(torch.load(f = "models/05_pytorch_going_modular_exercise_tinyvgg_model.pth"))
  loaded_model_0 = loaded_model_0.to(device)

  custom_image = torchvision.io.read_image(str(image_path)).type(torch.float32)
  custom_image = custom_image / 255

  custom_image_transform = transforms.Compose([
      transforms.Resize((64, 64))
  ])

  custom_image_transformed = custom_image_transform(custom_image)

  loaded_model_0.eval()
  with torch.inference_mode():
    custom_image_transformed_with_batch_size = custom_image_transformed.unsqueeze(dim = 0)
    custom_image_pred = loaded_model_0(custom_image_transformed_with_batch_size.to(device))

  custom_image_pred_probs = torch.softmax(custom_image_pred, dim = 1)
  custom_image_pred_label = torch.argmax(custom_image_pred_probs, dim = 1)
  data_classes = ['pizza', 'steak', 'sushi']

  print(f"Prediction probabilities: {custom_image_pred_probs}")
  print(f"\nPrediction label:{custom_image_pred_label}")
  print(f"\nPrediction class name: {data_classes[custom_image_pred_label]}")
  print(f"\nActual label: {image_path.parent.stem}")


Overwriting going_modular/predict.py


In [33]:
# Example running of predict.py
!python going_modular/predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

Image path:
  Image path: data/pizza_steak_sushi/test/sushi/175783.jpg
Prediction probabilities: tensor([[0.3610, 0.2880, 0.3511]], device='cuda:0')

Prediction label:tensor([0], device='cuda:0')

Prediction class name: pizza

Actual label: sushi
